In [ ]:
!nvidia-smi

In [ ]:
!pip install -q \
  transformers \
  accelerate \
  bitsandbytes \
  unsloth \
  google-api-python-client \
  google-auth \
  google-auth-oauthlib

**Mounting drive** (so don't have to download model everytime)

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

MODEL_DIR = "/content/drive/MyDrive/hf_models/llama3_1_8b_4bit"

In [ ]:
from google.colab import files
files.upload()  # upload credentials.json

In [ ]:
import os
from google_auth_oauthlib.flow import InstalledAppFlow
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from google.auth.transport.requests import Request

SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]
CREDENTIALS_FILE = "credentials.json"
TOKEN_FILE = "token.json"

def get_gmail_service_manual():
    creds = None

    if os.path.exists(TOKEN_FILE):
        creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)

    if not creds or not creds.valid:
        flow = InstalledAppFlow.from_client_secrets_file(
            CREDENTIALS_FILE,
            SCOPES,
            redirect_uri="urn:ietf:wg:oauth:2.0:oob"  # 🔑 critical
        )

        auth_url, _ = flow.authorization_url(
            prompt="consent",
            access_type="offline"
        )

        print("🔐 Open this URL in your browser:\n")
        print(auth_url)

        # ⬇️ YOU PASTE THE CODE HERE
        auth_code = input("\n📎 Paste the authorization code here: ").strip()

        flow.fetch_token(code=auth_code)
        creds = flow.credentials

        with open(TOKEN_FILE, "w") as f:
            f.write(creds.to_json())

    return build("gmail", "v1", credentials=creds)


In [ ]:
service = get_gmail_service_manual()
print("✅ Gmail service ready")

In [ ]:
%%writefile fetch_emails.py

from datetime import datetime, timedelta
import base64
import os
from google_auth_oauthlib.flow import InstalledAppFlow
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from google.auth.transport.requests import Request


SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]
CREDENTIALS_FILE = "credentials.json"
TOKEN_FILE = "token.json"

def get_gmail_service():
    creds = None

    if os.path.exists(TOKEN_FILE):
        creds = Credentials.from_authorized_user_file(
            TOKEN_FILE, SCOPES
        )

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            raise RuntimeError(
                "token.json missing or invalid. Run manual OAuth once."
            )

    return build("gmail", "v1", credentials=creds)

def get_last_12h_timestamp():
    return int((datetime.utcnow() - timedelta(hours=12)).timestamp())

def extract_body(payload):
    if "parts" in payload:
        for part in payload["parts"]:
            if part["mimeType"] == "text/plain":
                data = part["body"].get("data")
                if data:
                    return base64.urlsafe_b64decode(data).decode("utf-8", errors="ignore")
    data = payload["body"].get("data")
    if data:
        return base64.urlsafe_b64decode(data).decode("utf-8", errors="ignore")
    return ""

def normalize_message(msg, full_msg):
    headers = {h["name"]: h["value"] for h in full_msg["payload"]["headers"]}

    return {
        "id": msg["id"],
        "thread_id": msg["threadId"],
        "from": headers.get("From", ""),
        "subject": headers.get("Subject", ""),
        "date": headers.get("Date", ""),
        "labels": full_msg.get("labelIds", []),
        "body_text": extract_body(full_msg["payload"]),
    }

def fetch_last_12h_emails():
    service = get_gmail_service()
    after_ts = get_last_12h_timestamp()

    results = service.users().messages().list(
        userId="me",
        q=f"after:{after_ts}"
    ).execute()

    messages = results.get("messages", [])
    emails = []

    for msg in messages:
        full_msg = service.users().messages().get(
            userId="me", id=msg["id"], format="full"
        ).execute()

        emails.append(normalize_message(msg, full_msg))

    return emails


In [ ]:
from fetch_emails import fetch_last_12h_emails

print(fetch_last_12h_emails())

In [ ]:
!pip install -U bitsandbytes

In [ ]:
%%writefile model.py

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

HF_MODEL = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"

tokenizer = AutoTokenizer.from_pretrained(HF_MODEL)

model = AutoModelForCausalLM.from_pretrained(
    HF_MODEL,
    device_map="auto",
    torch_dtype=torch.float16
)

model.eval()
print("LLaMA model loaded.")


In [ ]:
%%writefile llama_chat.py

from model import model, tokenizer
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

MAX_INPUT_TOKENS = 2048
MAX_NEW_TOKENS = 200

def llama_chat(system_prompt, user_prompt):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=MAX_INPUT_TOKENS
    ).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=MAX_NEW_TOKENS,
            do_sample=False,
            repetition_penalty=1.15,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    generated = outputs[0][inputs["input_ids"].shape[-1]:]
    return tokenizer.decode(generated, skip_special_tokens=True).strip()


In [ ]:
%%writefile intent_classifier.py

from llama_chat import llama_chat

PRIMARY_LABELS = ["IGNORE", "READ_ONLY", "ACTION_REQUIRED"]
SPAM_LABELS = ["IGNORE", "ATTENTION_REQUIRED"]

def build_email_text(mail):
    return f"""From: {mail['from']}
Subject: {mail['subject']}

Body:
{mail['body_text'][:1200]}
"""

def classify_intent(mail, mailbox):
    prompt = (
        "Classify this email into IGNORE, READ_ONLY, ACTION_REQUIRED"
        if mailbox == "PRIMARY"
        else "Classify this email into IGNORE or ATTENTION_REQUIRED"
    )

    response = llama_chat(
        "You are an email intent classifier. Return ONLY the label.",
        f"{prompt}\n\n{build_email_text(mail)}"
    ).upper()

    for label in (PRIMARY_LABELS if mailbox == "PRIMARY" else SPAM_LABELS):
        if label in response:
            return label

    return "READ_ONLY"


In [ ]:
%%writefile summarizer.py

from llama_chat import llama_chat
from intent_classifier import build_email_text

def summarize_email(mail):
    return llama_chat(
    system_prompt=(
        "You summarize emails for a human user.\n\n"
        "Guidelines:\n"
        "- The email body may contain HTML, images, tracking links, or very little readable text.\n"
        "- If meaningful text exists, summarize it accurately.\n"
        "- If the body is mostly HTML, images, or boilerplate:\n"
        "  - Infer the purpose from subject, sender, and visible text\n"
        "  - Make a reasonable, conservative summary\n"
        "- Do NOT say \"not enough information\"\n"
        "- Do NOT mention HTML, images, or missing data\n"
        "- Do NOT hallucinate specific facts or details\n\n"
        "Use plain English.\n"
        "Be concise, helpful, and realistic."
    ),
    user_prompt=f"""Summarize this email in 3 to 4 bullet points.

{build_email_text(mail)}
"""
)


In [ ]:
%%writefile draft_generator.py

from llama_chat import llama_chat
from intent_classifier import build_email_text

def generate_draft(mail, intent, summary=None):
    if intent not in {"ACTION_REQUIRED", "READ_ONLY", "ATTENTION_REQUIRED"}:
        return None

    context = build_email_text(mail)
    if summary:
        context += f"\n\nSummary:\n{summary}"

    return llama_chat(
        "You write professional email replies.",
        f"{context}\n\nWrite a polite, professional reply."
    )


In [ ]:
emails = fetch_last_12h_emails()

for mail in emails:
    mailbox = "SPAM" if "SPAM" in mail["labels"] else "PRIMARY"

    intent = classify_intent(mail, mailbox)
    summary = summarize_email(mail)
    draft = generate_draft(mail, intent, summary)

    print("=" * 80)
    print(f"[{mailbox}] [{intent}] {mail['subject']}")
    print("From:", mail["from"])
    print("\nSUMMARY:\n", summary)

    if draft:
        print("\nDRAFT:\n", draft)


**FRONTEND**

**METHOD 2 ( STREAMLIT AND CLOUDFLARE)**

In [ ]:
!pip install -q streamlit cloudflared

In [ ]:
%%writefile app.py
import streamlit as st

from fetch_emails import fetch_last_12h_emails
from intent_classifier import classify_intent
from summarizer import summarize_email
from draft_generator import generate_draft
from llama_chat import llama_chat

st.set_page_config(
    page_title="MailEasy",
    layout="wide"
)

st.title("📬 MailEasy — AI Email Assistant")
st.markdown(
    "Fetch your last 12 hours of emails, understand intent, summarize, and draft replies using LLaMA."
)

# -----------------------------
# Main action button
# -----------------------------
if st.button("🚀 Fetch & Process Emails"):
    with st.spinner("Fetching emails and running AI..."):
        emails = fetch_last_12h_emails()

    if not emails:
        st.warning("No emails found in the last 12 hours.")
    else:
        st.success(f"Processed {len(emails)} emails")

        for idx, mail in enumerate(emails, start=1):
            mailbox = "SPAM" if "SPAM" in mail["labels"] else "PRIMARY"

            intent = classify_intent(mail, mailbox)
            summary = summarize_email(mail)
            draft = generate_draft(mail, intent, summary)

            with st.expander(f"📧 {idx}. {mail['subject']}"):
                st.markdown(f"**From:** {mail['from']}")
                st.markdown(f"**Mailbox:** `{mailbox}`")
                st.markdown(f"**Intent:** `{intent}`")

                st.subheader("📝 Summary")
                st.write(summary or "—")

                if draft:
                    st.subheader("✉️ Draft Reply")
                    st.text_area(
                        "Draft",
                        draft,
                        height=180,
                        key=f"draft_{idx}"
                    )
                else:
                    st.info("No reply required.")


In [ ]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

In [ ]:
# Restart Streamlit in the background, but let's monitor its output to ensure it starts without errors
# This time, we will not redirect stderr to /dev/null, so you can see any potential startup errors.
get_ipython().system_raw('streamlit run app.py --server.port 8501 --server.address 0.0.0.0 &')
print("Streamlit app launched in the background. Check the output above for any errors. If no errors, proceed to restart cloudflared.")

In [ ]:
!cloudflared tunnel --url http://localhost:8501


In [ ]:
# Stop any existing streamlit processes to ensure a clean restart
!pkill -f streamlit

In [ ]:
# Restart Streamlit in the background, but let's monitor its output to ensure it starts without errors
# This time, we will not redirect stderr to /dev/null, so you can see any potential startup errors.
get_ipython().system_raw('streamlit run app.py --server.port 8501 --server.address 0.0.0.0 &')
print("Streamlit app launched in the background. Check the output above for any errors. If no errors, proceed to restart cloudflared.")

In [ ]:
# Restart cloudflared tunnel after ensuring streamlit is running
!pkill -f cloudflared
!cloudflared tunnel --url http://localhost:8501